# A basic template to apply any models to train on the collected data

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from scipy.stats import zscore
from sklearn import preprocessing
pd.set_option('display.max_columns', None)
import graphviz
plt.rcParams['figure.figsize'] = [18, 10]
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split



In [2]:
trainDF = pd.read_csv('../outputs/trainXY.csv',parse_dates=['date_implement'])

# Shuffling the dataset to remove any prerequisite bias 
trainDF = trainDF.sample(frac=1)

In [3]:
trainDF.head()
trainDF = trainDF.dropna()

In [4]:
trainDF

,Unnamed: 0,Unnamed: 0_x,Unnamed: 0_x.1,Unnamed: 0.1,date_implement,policy,metric_change,state_x,policy_type,Unnamed: 0_y,stateName,CEN_stateCode,CEN_Bachelor's Degree or Higher,CEN_Employment Rate,CEN_Hispanic or Latino (of any race),CEN_Median Household Income,CEN_Total Employer Establishments,CEN_Total Households,CEN_Total Housing Units,CEN_Total Population,CEN_Unnamed: 0,CEN_Unnamed: 0.1,CEN_Without Health Care Coverage,CP_0_diff,CP_1_diff,CP_2_diff,FD_adminAndFireResponse,FD_fireResponseOnly,FD_lessThan10kCitizens,FD_moreThan10kCitizens,FD_totalFireDepartments,HB_bed_For-Profit,HB_bed_Non-Profit,HB_bed_State/Local Government,HB_bed_Total,OB_Prevalence,S1_people_fully_vaccinated,S1_people_fully_vaccinated_per_hundred,S1_people_vaccinated,S1_people_vaccinated_per_hundred,S2_people_fully_vaccinated,S2_people_fully_vaccinated_per_hundred,S2_people_vaccinated,S2_people_vaccinated_per_hundred,S3_people_fully_vaccinated,S3_people_fully_vaccinated_per_hundred,S3_people_vaccinated,S3_people_vaccinated_per_hundred,S4_people_fully_vaccinated,S4_people_fully_vaccinated_per_hundred,S4_people_vaccinated,S4_people_vaccinated_per_hundred,SP_Governor Political Affiliation,SP_State Attorney General Political Affiliation,SP_State House Majority Political Affiliation,SP_State Senate Majority Political Affiliation,Unnamed: 0_y.1,submission_date,state_y,new_case,caseInterpolate_MA7,caseInterpolate_savitzky31_3,caseInterpolate_gauss8,caseInterpolate_gauss3,new_case_zscore,Y_TREND_caseInterpolate_MA7_14,Y_TRENDQUANT_caseInterpolate_MA7_14,Y_SLOPE_caseInterpolate_MA7_14,Y_TREND_caseInterpolate_MA7_21,Y_TRENDQUANT_caseInterpolate_MA7_21,Y_SLOPE_caseInterpolate_MA7_21,Y_TREND_caseInterpolate_MA7_28,Y_TRENDQUANT_caseInterpolate_MA7_28,Y_SLOPE_caseInterpolate_MA7_28,Y_TREND_caseInterpolate_gauss8_14,Y_TRENDQUANT_caseInterpolate_gauss8_14,Y_SLOPE_caseInterpolate_gauss8_14,Y_TREND_caseInterpolate_gauss8_21,Y_TRENDQUANT_caseInterpolate_gauss8_21,Y_SLOPE_caseInterpolate_gauss8_21,Y_TREND_caseInterpolate_gauss8_28,Y_TRENDQUANT_caseInterpolate_gauss8_28,Y_SLOPE_caseInterpolate_gauss8_28,Y_TREND_caseInterpolate_gauss3_14,Y_TRENDQUANT_caseInterpolate_gauss3_14,Y_SLOPE_caseInterpolate_gauss3_14,Y_TREND_caseInterpolate_gauss3_21,Y_TRENDQUANT_caseInterpolate_gauss3_21,Y_SLOPE_caseInterpolate_gauss3_21,Y_TREND_caseInterpolate_gauss3_28,Y_TRENDQUANT_caseInterpolate_gauss3_28,Y_SLOPE_caseInterpolate_gauss3_28
859,859,859,859,2,2020-08-17,C1_School closing,-1.0,Pennsylvania,0,37.0,Pennsylvania,PA,32.3,60.0,1049615.0,63463.0,303224.0,5119249.0,5742828.0,13002700.0,38.0,0.0,5.8,125.0,366.0,659.0,1675.0,240.0,1915.0,292.0,2207.0,0.28,2.49,0.00,2.77,31.5,403898.0,3.15,1269266.0,9.91,4629086.0,36.16,6727935.0,52.55,6853123.0,53.53,8617751.0,67.32,7430988.0,58.05,10349983.0,80.85,1.0,1.0,0.0,0.0,40144.0,2020-08-17,PA,457.0,806.428571,599.629958,739.0,715.0,-0.804104,-1.0,-3.826531,-6.967473,-1.0,-6.768707,-6.967473,-1.0,-2.744898,-6.967473,-1.0,-7.071429,-6.299231,-1.0,-6.476190,-6.299231,-1.0,-4.464286,-6.299231,-1.0,-6.214286,-8.020769,-1.0,-10.523810,-8.020769,-1.0,-6.178571,-8.020769
594,594,594,594,22,2020-06-04,H6_Facial Coverings,2.0,Mississippi,0,24.0,Mississippi,MS,22.3,52.6,105220.0,45792.0,59130.0,1100229.0,1319945.0,2961279.0,36.0,0.0,13.0,220.0,387.0,615.0,459.0,8.0,467.0,62.0,529.0,0.79,1.23,1.64,3.66,39.7,114284.0,3.84,312244.0,10.49,747819.0,25.13,951726.0,31.98,1064808.0,35.78,1289365.0,43.32,1391610.0,46.76,1590255.0,53.43,0.0,0.0,0.0,0.0,9929.0,2020-06-04,MS,255.0,298.142857,726.354119,364.0,318.0,-0.604144,-1.0,-0.214286,2.985055,1.0,1.585034,2.985055,1.0,1.056122,2.985055,1.0,4.214286,3.322308,1.0,3.761905,3.322308,1.0,3.392857,3.322308,1.0,0.714286,2.252308,1.0,1.809524,2.252308,1.0,1.678571,2.252308
1491,1491,1491,371,9,2021-06-06,H3_Contact tracing,-1.0,Montana,0,26.0,Montana,MT,33.6,59.8,45199.0,57153.0,38959.0,437651.0,514803.0,1084225.0,20.0,0.0,8.3,111.0,341.0,661.0,313.0,5.0,318.0,29.0,347.0,0.22,3.04,0.12,3.38,28.5,48739.0,4.56,127657.0,11.9

In [5]:
# Finalizing Indexes so we can drop state names 
trainDF = trainDF.reset_index(drop=True)

# Get a list of columns with features 
columns = trainDF.columns.to_list()

In [6]:
Y_columns = [x for x in columns if 'Y_' in x]

In [7]:
X_columns = [x for x in columns if 'Y_' not in x and 'Unnamed' not in x]

X_columns.remove('date_implement')
# X_columns.remove('policy')
X_columns.remove('state_x')
X_columns.remove('policy_type')
X_columns.remove('stateName')
X_columns.remove('CEN_stateCode')
X_columns.remove('submission_date')
X_columns.remove('state_y')

X_columns.remove('caseInterpolate_gauss3')
X_columns.remove('caseInterpolate_MA7')
X_columns.remove('caseInterpolate_savitzky31_3')
X_columns.remove('caseInterpolate_gauss8')
X_columns.remove('new_case_zscore')
X_columns.remove('new_case')



In [8]:
X = trainDF[X_columns]
y = trainDF[Y_columns]


In [9]:
y

,Y_TREND_caseInterpolate_MA7_14,Y_TRENDQUANT_caseInterpolate_MA7_14,Y_SLOPE_caseInterpolate_MA7_14,Y_TREND_caseInterpolate_MA7_21,Y_TRENDQUANT_caseInterpolate_MA7_21,Y_SLOPE_caseInterpolate_MA7_21,Y_TREND_caseInterpolate_MA7_28,Y_TRENDQUANT_caseInterpolate_MA7_28,Y_SLOPE_caseInterpolate_MA7_28,Y_TREND_caseInterpolate_gauss8_14,Y_TRENDQUANT_caseInterpolate_gauss8_14,Y_SLOPE_caseInterpolate_gauss8_14,Y_TREND_caseInterpolate_gauss8_21,Y_TRENDQUANT_caseInterpolate_gauss8_21,Y_SLOPE_caseInterpolate_gauss8_21,Y_TREND_caseInterpolate_gauss8_28,Y_TRENDQUANT_caseInterpolate_gauss8_28,Y_SLOPE_caseInterpolate_gauss8_28,Y_TREND_caseInterpolate_gauss3_14,Y_TRENDQUANT_caseInterpolate_gauss3_14,Y_SLOPE_caseInterpolate_gauss3_14,Y_TREND_caseInterpolate_gauss3_21,Y_TRENDQUANT_caseInterpolate_gauss3_21,Y_SLOPE_caseInterpolate_gauss3_21,Y_TREND_caseInterpolate_gauss3_28,Y_TRENDQUANT_caseInterpolate_gauss3_28,Y_SLOPE_caseInterpolate_gauss3_28
0,-1.0,-3.826531,-6.967473,-1.0,-6.768707,-6.967473,-1.0,-2.744898,-6.967473,-1.0,-7.071429,-6.299231,-1.0,-6.476190,-6.299231,-1.0,-4.464286,-6.299231,-1.0,-6.214286,-8.020769,-1.0,-10.523810,-8.020769,-1.0,-6.178571,-8.020769
1,-1.0,-0.214286,2.985055,1.0,1.585034,2.985055,1.0,1.056122,2.985055,1.0,4.214286,3.322308,1.0,3.761905,3.322308,1.0,3.392857,3.322308,1.0,0.714286,2.252308,1.0,1.809524,2.252308,1.0,1.678571,2.252308
2,1.0,0.112245,-1.712418,-1.0,-1.578231,-1.712418,-1.0,-1.862245,-1.712418,-1.0,-0.857143,-1.284615,-1.0,-1.238095,-1.284615,-1.0,-1.535714,-1.284615,1.0,0.285714,-0.880000,-1.0,-0.714286,-0.880000,-1.0,-1.357143,-0.880000
3,1.0,3.540816,5.788242,1.0,5.557823,5.788242,1.0,5.081633,5.788242,1.0,4.000000,4.707692,1.0,4.523810,4.707692,1.0,4.642857,4.707692,1.0,2.928571,4.858462,1.0,4.619048,4.858462,1.0,5.000000,4.858462
4,1.0,109.306122,113.271758,1.0,107.632653,113.271758,1.0,104.785714,113.271758,1.0,62.428571,83.254615,1.0,75.523810,83.254615,1.0,80.107143,83.254615,1.0,81.857143,103.574615,1.0,88.714286,103.574615,1.0,92.571429,103.574615
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1578,1.0,34.969388,37.091648,1.0,34.000000,37.091648,1.0,26.005102,37.091648,1.0,22.500000,29.358462,1.0,26.380952,29.358462,1.0,27.357143,29.358462,1.0,26.428571,40.135385,1.0,42.095238,40.135385,1.0,28.607143,40.135385
1579,-1.0,-9.489796,-19.854176,-1.0,-16.925170,-19.854176,-1.0,-32.214286,-19.854176,-1.0,-9.785714,-18.688462,-1.0,-16.285714,-18.688462,-1.0,-24.214286,-18.688462,-1.0,-9.000000,-13.749231,-1.0,-10.619048,-13.749231,-1.0,-23.500000,-13.749231
1580,-1.0,-6.000000,-4.273407,-1.0,-3.734694,-4.273407,-1.0,-8.744898,-4.273407,-1.0,-3.285714,-4.190000,-1.0,-4.000000,-4.190000,-1.0,-4.000000,-4.190000,-1.0,-3.642857,-3.729231,-1.0,-4.000000,-3.729231,-1.0,-7.035714,-3.729231
1581,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,1.0,1.142857,0.400769,1.0,0.761905,0.400769,1.0,0.571429,0.400769,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000


# Encoding Policy Names 

In [10]:
X['policy'].unique()

array(['C1_School closing', 'H6_Facial Coverings', 'H3_Contact tracing',
       'C2_Workplace closing', 'C3_Cancel public events',
       'C4_Restrictions on gatherings', 'C6_Stay at home requirements',
       'H2_Testing policy', 'H7_Vaccination policy',
       'C5_Close public transport',
       'C7_Restrictions on internal movement',
       'C8_International travel controls',
       'H1_Public information campaigns'], dtype=object)

In [11]:
le = preprocessing.LabelEncoder()
le.fit(X['policy'])
print(le.classes_)

['C1_School closing' 'C2_Workplace closing' 'C3_Cancel public events'
 'C4_Restrictions on gatherings' 'C5_Close public transport'
 'C6_Stay at home requirements' 'C7_Restrictions on internal movement'
 'C8_International travel controls' 'H1_Public information campaigns'
 'H2_Testing policy' 'H3_Contact tracing' 'H6_Facial Coverings'
 'H7_Vaccination policy']


In [12]:
X['policy'] = pd.Series(le.transform(X['policy']))
original_policies_decoded = pd.Series(le.inverse_transform(X['policy']))

/var/folders/66/w1588p9n1654b1yky4q5nz580000gn/T/ipykernel_9042/2867200982.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['policy'] = pd.Series(le.transform(X['policy']))


------

-------

# Model Input Specification

In [13]:
X = X.apply(zscore)
y = y.apply(zscore)

In [14]:
y.head()

,Y_TREND_caseInterpolate_MA7_14,Y_TRENDQUANT_caseInterpolate_MA7_14,Y_SLOPE_caseInterpolate_MA7_14,Y_TREND_caseInterpolate_MA7_21,Y_TRENDQUANT_caseInterpolate_MA7_21,Y_SLOPE_caseInterpolate_MA7_21,Y_TREND_caseInterpolate_MA7_28,Y_TRENDQUANT_caseInterpolate_MA7_28,Y_SLOPE_caseInterpolate_MA7_28,Y_TREND_caseInterpolate_gauss8_14,Y_TRENDQUANT_caseInterpolate_gauss8_14,Y_SLOPE_caseInterpolate_gauss8_14,Y_TREND_caseInterpolate_gauss8_21,Y_TRENDQUANT_caseInterpolate_gauss8_21,Y_SLOPE_caseInterpolate_gauss8_21,Y_TREND_caseInterpolate_gauss8_28,Y_TRENDQUANT_caseInterpolate_gauss8_28,Y_SLOPE_caseInterpolate_gauss8_28,Y_TREND_caseInterpolate_gauss3_14,Y_TRENDQUANT_caseInterpolate_gauss3_14,Y_SLOPE_caseInterpolate_gauss3_14,Y_TREND_caseInterpolate_gauss3_21,Y_TRENDQUANT_caseInterpolate_gauss3_21,Y_SLOPE_caseInterpolate_gauss3_21,Y_TREND_caseInterpolate_gauss3_28,Y_TRENDQUANT_caseInterpolate_gauss3_28,Y_SLOPE_caseInterpolate_gauss3_28
0,-1.334114,-0.079294,-0.112641,-1.280944,-0.111249,-0.112641,-1.284388,-0.041253,-0.112641,-1.334231,-0.129180,-0.115627,-1.332367,-0.120354,-0.115627,-1.330543,-0.081642,-0.115627,-1.351021,-0.104208,-0.136167,-1.307919,-0.179911,-0.136167,-1.296008,-0.108095,-0.136167
1,-1.334114,-0.031627,0.049814,0.831279,0.028365,0.049814,0.829111,0.025232,0.049814,0.771540,0.057871,0.055054,0.770461,0.061714,0.055054,0.771502,0.067632,0.055054,0.796234,-0.014788,0.023223,0.823255,0.015143,0.023223,0.829982,0.029536,0.023223
2,0.799121,-0.027318,-0.026863,-1.280944,-0.024502,-0.026863,-1.284388,-0.025814,-0.026863,-1.334231,-0.026183,-0.026670,-1.332367,-0.027203,-0.026670,-1.330543,-0.026004,-0.026670,0.796234,-0.020319,-0.025376,-1.307919,-0.024771,-0.025376,-1.296008,-0.023640,-0.025376
3,0.799121,0.017926,0.095570,0.831279,0.094762,0.095570,0.829111,0.095644,0.095570,0.771540,0.054319,0.079630,0.770461,0.075263,0.079630,0.771502,0.091380,0.079630,0.796234,0.013790,0.063658,0.823255,0.059577,0.063658,0.829982,0.087716,0.063658
4,0.799121,1.413602,1.850017,0.831279,1.800719,1.850017,0.829111,1.839609,1.850017,0.771540,1.022721,1.473010,0.770461,1.337882,1.473010,0.771502,1.525089,1.473010,0.796234,1.032441,1.595274,0.823255,1.389560,1.595274,0.829982,1.621677,1.595274


In [15]:
X.head()

,policy,metric_change,CEN_Bachelor's Degree or Higher,CEN_Employment Rate,CEN_Hispanic or Latino (of any race),CEN_Median Household Income,CEN_Total Employer Establishments,CEN_Total Households,CEN_Total Housing Units,CEN_Total Population,CEN_Without Health Care Coverage,CP_0_diff,CP_1_diff,CP_2_diff,FD_adminAndFireResponse,FD_fireResponseOnly,FD_lessThan10kCitizens,FD_moreThan10kCitizens,FD_totalFireDepartments,HB_bed_For-Profit,HB_bed_Non-Profit,HB_bed_State/Local Government,HB_bed_Total,OB_Prevalence,S1_people_fully_vaccinated,S1_people_fully_vaccinated_per_hundred,S1_people_vaccinated,S1_people_vaccinated_per_hundred,S2_people_fully_vaccinated,S2_people_fully_vaccinated_per_hundred,S2_people_vaccinated,S2_people_vaccinated_per_hundred,S3_people_fully_vaccinated,S3_people_fully_vaccinated_per_hundred,S3_people_vaccinated,S3_people_vaccinated_per_hundred,S4_people_fully_vaccinated,S4_people_fully_vaccinated_per_hundred,S4_people_vaccinated,S4_people_vaccinated_per_hundred,SP_Governor Political Affiliation,SP_State Attorney General Political Affiliation,SP_State House Majority Political Affiliation,SP_State Senate Majority Political Affiliation
0,-1.264257,-0.778684,0.027266,-0.114987,-0.072377,-0.144221,0.862718,1.082603,1.046093,0.894392,-0.936485,-0.950226,-0.264931,0.620350,3.900652,2.011442,4.065500,2.364285,3.950312,-0.296944,0.919076,-0.836799,0.342353,-0.197087,0.622027,-0.878260,0.837483,-0.754867,0.859170,-0.077782,1.036157,0.776957,0.979441,0.483325,1.058773,1.127021,0.856773,0.163936,1.146877,1.591399,1.090637,1.065975,-0.730767,-0.701809
1,1.519922,1.195673,-1.857525,-2.069971,-0.411303,-1.798374,-0.560253,-0.521203,-0.501610,-0.488024,1.430133,0.480617,0.352808,-0.372199,0.123554,-0.450536,-0.007534,-0.406077,-0.087572,1.379854,-0.729340,2.794509,1.644365,1.897997,-0.600180,-0.434775,-0.573567,-0.501484,-0.592394,-1.996202,-0.574249,-1.819169,-0.580787,-1.647807,-0.553617,-1.605401,-0.537126,-1.252534,-0.526253,-1.286988,-0.916896,-0.938108,-0.730767,-0.701809
2,1.266815,-0.778684,0.272289,-0.167825,-0.432843,-0.734890,-0.677842,-0.785608,-0.783354,-0.746440,-0.114743,-1.161087,-1.000334,0.665466,-0.329946,-0.482372,-0.426652,-0.803564,-0.525530,-0.494214,1.638623,-0.571093,1.234743,-0.963581,-0.876788,0.027991,-0.845725,0.131974,-0.723187,0.111799,-0.707956,-0.279415,-0.738656,-0.561230,-0.718635,-0.792505,-0.732045,-0.696736,-0.711406,-0.797809,-0.916896,-0.938108,-0.730767,-0.701809
3,1.519922,0.537554,-1.857525,-2.069971,-0.411303,-1.798374,-0.560253,-0.521203,-0.501610,-0.488024,1.430133,0.480617,0.352808,-0.372199,0.123554,-0.450536,-0.007534,-0.406077,-0.087572,1.379854,-0.729340,2.794509,1.644365,1.897997,-0.600180,-0.434775,-0.573567,-0.501484,-0.592394,-1.996202,-0.574249,-1.819169,-0.580787,-1.647807,-0.553617,-1.605401,-0.537126,-1.252534,-0.526253,-1.286988,-0.916896,-0.938108,-0.730767,-0.701809
4,-1.264257,-0.778684,-0.990521,0.202037,-0.250176,-0.692766,-0.036830,0.076394,0.059410,0.038466,0.016736,-0.709242,-0.529676,-0.011272,0.651602,-0.429313,0.476279,0.774338,0.562147,0.031840,0.186447,0.048886,0.254577,1.157052,0.631712,0.972806,0.270332,0.656214,-0.011242,-0.420419,-0.075639,-0.718624,-0.047725,-0.518007,-0.114595,-0.982636,-0.073241,-0.779541,-0.116112,-1.067592,-0.916896,-0.938108,-0.730767,-0.701809


In [16]:
Y_SELECTED = 'Y_TRENDQUANT_caseInterpolate_MA7_21'

In [17]:
y_selected = y[[Y_SELECTED]]

# Models

In [19]:
from sklearn.kernel_ridge import KernelRidge
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_absolute_error

from sklearn import tree

## 1 - Decision Tree Regressor

In [20]:

clf = tree.DecisionTreeRegressor(max_depth=10,splitter='random')
clf.fit(X[:1500], y_selected[:1500])

X_train = X[:1500]
y_train = y_selected[:1500]
y_train_predict=clf.predict(X_train)

X_test = X[1500:]
y_test = y_selected[1500:]
y_test_predict=clf.predict(X_test)

train_error = mean_absolute_error(y_train, y_train_predict)
test_error = mean_absolute_error(y_test, y_test_predict)

print(f'train error: {train_error}\ntest_error: {test_error}')
print('\n')

train error: 0.29988040214878414
test_error: 0.45742003528225633




## Kernel Ridge

In [21]:
alpha = [10,40,100,10,50,10,40,100]
degree = [3,3,3,5,5,7,7,7]
for i in range (7):
    
    krr = KernelRidge(alpha = alpha[i] ,degree = degree[i])
    krr.fit(X[:1500], y_selected[:1500])

    X_train = X[:1500]
    y_train = y_selected[:1500]
    y_train_predict=krr.predict(X_train)


    X_test = X[1500:]
    y_test = y_selected[1500:]
    y_test_predict=krr.predict(X_test)

    train_error = mean_absolute_error(y_train, y_train_predict)
    test_error = mean_absolute_error(y_test, y_test_predict)

    print(f'alpha: {alpha[i]}, degree: {degree[i]}\ntrain error: {train_error}\ntest_error: {test_error}')
    print('\n')


alpha: 10, degree: 3
train error: 0.4110513767605823
test_error: 0.3742977058971324


alpha: 40, degree: 3
train error: 0.4098354094794187
test_error: 0.36667563157393407


alpha: 100, degree: 3
train error: 0.4088059234179802
test_error: 0.36082073430919803


alpha: 10, degree: 5
train error: 0.4110513767605823
test_error: 0.3742977058971324


alpha: 50, degree: 5
train error: 0.409725513650924
test_error: 0.3652813753686342


alpha: 10, degree: 7
train error: 0.4110513767605823
test_error: 0.3742977058971324


alpha: 40, degree: 7
train error: 0.4098354094794187
test_error: 0.36667563157393407




### Creating the analysis


#### State Analysis

In [22]:
df_temp = pd.DataFrame(y_train_predict,columns=[f'predict_{Y_SELECTED}'])

In [23]:
result = pd.concat([df_temp, y_train,trainDF[['policy','state_x','metric_change']][:1500] ], axis=1)

In [24]:
result['prediction_error'] = pow(result[f'predict_{Y_SELECTED}'] - result[Y_SELECTED],2)

In [25]:
result.sort_values(by='prediction_error')[0:50]

,predict_Y_TRENDQUANT_caseInterpolate_MA7_21,Y_TRENDQUANT_caseInterpolate_MA7_21,policy,state_x,metric_change,prediction_error
189,0.003410,0.003353,C3_Cancel public events,Nevada,1.0,3.219483e-09
746,0.001743,0.001875,H2_Testing policy,Vermont,-1.0,1.733939e-08
401,-0.261758,-0.262006,C4_Restrictions on gatherings,Connecticut,-3.0,6.145218e-08
362,0.068936,0.068612,H6_Facial Coverings,Colorado,1.0,1.044425e-07
668,-0.022351,-0.022683,C6_Stay at home requirements,South Carolina,-1.0,1.102498e-07
112,0.110134,0.110679,C6_Stay at home requirements,Arizona,2.0,2.963812e-07
35,0.010466,0.011084,C5_Close public transport,Maine,-1.0,3.814134e-07
1396,-0.116389,-0.117048,C1_School closing,New Hampshire,-1.0,4.343189e-07
819,0.003628,0.002443,C3_Cancel public events,Delaware,1.0,1.404131e-06
688,0.121056,0.119774,C5_Close public transport,Tennessee,1.0,1.643414e-06


In [26]:
table_states = result['state_x'].unique()

In [27]:
ERROR_AR = []
for state in table_states:
    temp = {}
    state_mean_error = result[result['state_x']==state]['prediction_error'].mean() *100
    temp = {
        'state':f'{state}',
        'error':state_mean_error
    }
    ERROR_AR.append(temp)

In [28]:
ERROR_AR = sorted(ERROR_AR, key = lambda i: i['error'])

In [29]:
ERROR_AR

[{'state': 'West Virginia', 'error': 1.3220885270672524},
 {'state': 'Delaware', 'error': 1.6935025989970982},
 {'state': 'Vermont', 'error': 1.69609869558206},
 {'state': 'Wyoming', 'error': 1.992805197517319},
 {'state': 'New Hampshire', 'error': 2.020406177993005},
 {'state': 'Rhode Island', 'error': 2.18541136036482},
 {'state': 'Alaska', 'error': 2.5919947170837303},
 {'state': 'Maine', 'error': 2.8851478167797167},
 {'state': 'Hawaii', 'error': 2.9445374042351387},
 {'state': 'Montana', 'error': 2.9800582711181205},
 {'state': 'Kansas', 'error': 3.096506791373912},
 {'state': 'Kentucky', 'error': 3.266360495073315},
 {'state': 'South Dakota', 'error': 4.274524700009742},
 {'state': 'Idaho', 'error': 4.387822549953334},
 {'state': 'North Dakota', 'error': 4.498625110917682},
 {'state': 'Connecticut', 'error': 4.804661341451104},
 {'state': 'Oregon', 'error': 5.692894731473625},
 {'state': 'New Mexico', 'error': 7.313957076365071},
 {'state': 'Nevada', 'error': 7.445059612343143},


# Apply model here

In [30]:
from sklearn.kernel_ridge import KernelRidge
import numpy as np



krr = KernelRidge(alpha=50.0)
krr.fit(X[:1500], y_selected[:1500])

KernelRidge(alpha=50.0)

In [31]:
T = pd.DataFrame(krr.predict(X[1500:]))

In [32]:
model_output = pd.DataFrame(krr.predict(X[1500:]))
actual_value = y_selected[1500:].reset_index().apply(zscore)
actual_value = actual_value.drop(columns=['index'])

In [33]:
result = pd.concat([actual_value, model_output], axis=1)

In [34]:
result.columns

Index(['Y_TRENDQUANT_caseInterpolate_MA7_21', 0], dtype='object')

In [35]:
result['diff'] = result['Y_SLOPE_caseInterpolate_gauss3_21'] - result[0]

KeyError: 'Y_SLOPE_caseInterpolate_gauss3_21'

In [ ]:
result['diff'].mean()

In [ ]:
0.03625768498035307

In [ ]:
ax = model_output.plot() 


actual_value.plot(ax=ax)

In [ ]:
ax = model_output.plot() 


actual_value.plot(ax=ax)

In [ ]:
T.plot()
y_selected[1500:].plot()